In [ ]:
from pyspark.sql.functions import to_date, first, col, round

In [ ]:
df_junto = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")

In [ ]:
moedas = ['USD', 'EUR', 'GBP']

df_moedas = df_junto.filter(df_junto.moeda.isin(moedas))

In [ ]:
df_moedas = df_moedas.withColumn("data", to_date("data"))

In [ ]:
resultado_taxas_conversao = df_moedas.groupBy("data")\
                                    .pivot("moeda")\
                                    .agg(first("taxa"))\
                                    .orderBy("data", ascending=False)

In [ ]:
resultado_valores_reais = resultado_taxas_conversao.select("*")

+----------+--------+--------+--------+
|      data|     EUR|     GBP|     USD|
+----------+--------+--------+--------+
|2023-04-01|0.181681|0.159983|0.197499|
|2023-04-02|0.182756|0.160693| 0.19748|
|2023-04-03|0.180972|0.158945|0.197449|
|2023-04-04|0.179912|0.157752|0.197153|
|2023-04-05|0.182154|0.159374|0.198622|
+----------+--------+--------+--------+
only showing top 5 rows



In [ ]:
for moeda in moedas:
    resultado_valores_reais = resultado_valores_reais\
                                .withColumn(
                                    moeda, round(1/col(moeda), 4)  
                                )

In [ ]:
resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [ ]:
# Salvando arquivo de taxas

resultado_taxas_conversao.write\
  .mode("overwrite")\
  .format("csv")\
  .option("header", "true")\
  .save("dbfs:/databricks-results/prata/taxas_conversao")


resultado_valores_reais.write\
  .mode("overwrite")\
  .format("csv")\
  .option("header", "true")\
  .save("dbfs:/databricks-results/prata/valores_reais")

In [ ]:
display(dbutils.fs.ls("dbfs:/databricks-results/bronze/2023/06/02"))